### 第一步：準備 YOLO 模型

在使用 YOLO 模型時，應該使用 **預訓練權重（`.pt`）** 進行推理或訓練優化，而 **`.yaml` 檔案** 是用來定義模型結構和訓練數據集的配置。

具體來說：
- **`.pt` 文件** 是你已經訓練好的模型權重，包含了模型參數，用來進行推理、微調等任務。這是進行剪枝、量化等優化的主要對象。
- **`.yaml` 文件** 是 YOLO 的配置檔案，主要用來定義模型架構（如 YOLOv5 的不同大小版本）以及訓練時使用的數據集、類別等。

---

In [ ]:
import torch

# 加載你自己訓練的 YOLO 模型
model = torch.hub.load('ultralytics/yolov5', 'custom', path=r'D:\Learning_Python\30-Day_AI_Deep_Learning_Plan\yolov5-master\weights\best.pt')

在模型優化中，我們可以採取以下幾個技術步驟來提升效能：

### 第二步：模型剪枝 (Pruning)
**目的：** 
剪枝的目的是移除不必要的神經元或卷積核，減少模型的參數量，以降低推理的計算成本和記憶體佔用，從而提升推理效能，特別適合資源有限的設備。

### 操作步驟：
1. **瞭解剪枝技術：** 剪枝會根據某些策略移除重要性較低的參數。常見的策略包括：基於稀疏性的剪枝（如修剪權重值接近零的參數）或者基於效能的剪枝（如計算每個參數對最終預測結果的貢獻）。
2. **選擇合適的剪枝工具：** 可以使用 PyTorch 或第三方庫（例如 Torch-Pruning 或 AutoML 工具）來進行模型剪枝。
3. **實際代碼範例：**

**為什麼這麼做？**
剪枝能夠有效減少模型的冗餘參數，尤其是在訓練過程中可能導致過擬合的部分。透過剪枝，你可以讓模型在推理時更快、資源佔用更少，適合應用在嵌入式系統或雲端上的大規模佈署。

這一步完成後，請測試模型的效能變化，看看在削減部分參數後是否還能保持準確率，並檢查推理速度是否提升。

---

In [ ]:
import torch_pruning as tp

# 設定模型剪枝
strategy = tp.strategy.L1Strategy()  # 基於 L1 正則化進行剪枝
model = model.eval()  # 將模型設為評估模式

# 確定要剪枝的層，例如卷積層
layer_to_prune = model.model[4]  # 假設你想剪枝第 4 層卷積層
importance = strategy(layer_to_prune.weight)  # 計算該層的重要性

# 剪枝層
pruner = tp.pruner.MagnitudePruner(model, [layer_to_prune], importance)
pruner.step()  # 執行剪枝

### 測試剪枝後模型效能的步驟：

#### 1. 測試推理速度
剪枝的目的是加速模型的推理，因此一個重要的效能指標就是推理速度。我們可以通過計算模型在一組測試圖像上的平均推理時間來評估。

In [ ]:
import time
import torch

# 測試用的圖像數據（可以使用一張或多張測試圖像）
img = torch.randn(1, 3, 640, 640)  # 模擬一張 640x640 大小的隨機圖像

# 測試推理速度
start_time = time.time()
with torch.no_grad():  # 禁止梯度計算來加速推理
    for _ in range(100):  # 假設我們測試 100 次推理
        _ = model(img) # 進行推理
end_time = time.time() # 計算推理時間

average_inference_time = (end_time - start_time) / 100
print(f"剪枝後模型的平均推理時間: {average_inference_time:.4f} 秒")

#### 2. 測試模型準確性
剪枝可能會影響模型的準確性，因此需要通過測試集來檢測模型的準確性。我們可以通過比較剪枝前後模型的準確率來評估這一點。

In [ ]:
# 假設你有一個測試數據集 test_loader
correct = 0
total = 0

# 將模型設置為評估模式
model.eval()

with torch.no_grad():  # 禁止梯度計算來加速推理
    for images, labels in test_loader:  # 迭代測試集中的每個批次
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"剪枝後模型的準確率: {accuracy:.4f}")

#### 3. 測試內存使用情況
可以通過監控模型的內存佔用情況來檢查剪枝的效果。在 PyTorch 中，我們可以使用 `torch.cuda.memory_allocated()` 來檢查 GPU 內存的佔用，或者通過 `psutil` 庫來監控 CPU 內存。

In [ ]:
import torch
import psutil

# 查看 GPU 內存使用情況
if torch.cuda.is_available():
    print(f"剪枝後的 GPU 內存佔用: {torch.cuda.memory_allocated() / 1024 ** 2:.2f} MB")

# 查看 CPU 內存使用情況
process = psutil.Process()
print(f"剪枝後的 CPU 內存佔用: {process.memory_info().rss / 1024 ** 2:.2f} MB")

#### 4. 比較剪枝前後效能差異
在進行完剪枝後的測試後，應該將這些結果與剪枝前的效能進行對比，從而得出剪枝是否有效的結論。如果推理速度加快而準確率僅有小幅下降，則說明剪枝的效果比較好。 

**為什麼這麼做？**
測試推理速度能夠衡量模型運行時的性能提升，測試準確性可以幫助確定模型效能是否受損，而內存的佔用可以讓你確定剪枝後的資源利用率是否提升。這些綜合評估可以幫助你在剪枝後選擇更高效的模型。

---

### 第三步：量化（Quantization）

在測試完剪枝後模型的效能之後，下一步你可以考慮進行 **量化（Quantization）**，這是一種常見的優化技術，可以進一步減少模型的計算量和內存佔用，而不會對模型的準確性造成太大的影響。

### 量化的概念
量化是一種將模型參數（通常是浮點數）轉換為較低精度（例如整數）的技術，從而減少模型的存儲空間並加速推理過程。常見的量化技術包括 **靜態量化（Post-Training Quantization, PTQ）** 和 **動態量化（Dynamic Quantization）**。


### 動態量化的特點：
- **應用場景：** 動態量化主要用來減少推理時的計算量和記憶體佔用，特別適合在 **CPU** 上運行的模型。常見用於全連接層（如 `torch.nn.Linear`）的量化，適合推理場景。
- **工作原理：** 動態量化會在推理時將模型的權重量化（通常轉換為 8 位整數），但輸入資料仍然保持浮點格式。在推理過程中，模型的權重會動態從浮點格式轉換為量化格式。
- **使用方法：** 代碼中的 `torch.quantization.quantize_dynamic()` 就是典型的動態量化方法，主要對全連接層進行量化，這有助於減少推理的資源佔用和時間。

```python
quantized_model = torch.quantization.quantize_dynamic(
    model,  # 目標模型
    {torch.nn.Linear},  # 只對全連接層進行量化
    dtype=torch.qint8  # 使用8位量化
)
```

### **動態量化 vs 靜態量化：**
1. **動態量化（Dynamic Quantization）：**
   - 只對模型的權重進行量化，而輸入和中間激活值依然保持浮點格式。
   - 優點是使用簡單，可以直接對已經訓練好的模型應用，不需要重新訓練或準備額外的數據。
   - 主要適用於 **CPU** 上推理速度的提升。

2. **靜態量化（Static Quantization）：**
   - 靜態量化不僅量化模型的權重，還量化激活值（即中間的輸出），因此可以帶來更大的效能提升。
   - 靜態量化需要在量化之前使用一些校準數據（calibration data）來確定輸入和激活的量化範圍。
   - 需要額外的校準步驟，這通常需要一些樣本來計算激活值的範圍。

### 動態量化的優勢：
- **簡單易用：** 不需要額外的校準數據，也不需要重新訓練，適合對已訓練完的模型進行部署。
- **推理加速：** 能顯著減少推理時的計算量，特別是在 **CPU** 上有明顯的效能提升。

### 靜態量化的應用場景：
如果你對推理效能的要求極高，或者希望將模型存儲空間最小化，靜態量化會比動態量化更有效。不過，它需要額外的數據準備（校準數據）以及更多的配置工作。

總結：當前實現的量化屬於 **動態量化**，適合大多數場景，特別是對已經訓練完成的模型，不需要重新訓練或校準數據的情況。如果你對性能有更高的需求，或者需要在資源受限的設備上部署模型，可以進一步研究 **靜態量化** 或 **混合精度訓練**。

### **動態量化（Dynamic Quantization）測試**

#### **可以將量化前後的模型測試分成兩個獨立的部分，分別進行量化前的推理測試和量化後的推理測試。**

#### 1. 量化前的推理時間與效能測驗：

In [ ]:
import torch
import time

# 定義一個函數來測試模型的推理時間
def test_inference_time(model, img, iterations=100):
    start_time = time.time() # 記錄開始時間
    with torch.no_grad(): # 禁止梯度計算來加速推理
        for _ in range(iterations): # 假設我們測試 100 次推理
            _ = model(img) # 進行推理
    end_time = time.time() # 記錄結束時間
    return (end_time - start_time) / iterations # 計算平均推理時間

# 模擬一張 640x640 大小的隨機圖像
img = torch.randn(1, 3, 640, 640)  # 模擬一張 640x640 大小的隨機圖像

# 加載你的模型
model = torch.load('path_to_your_model.pt')

# 測試模型的推理時間
inference_time_before = test_inference_time(model, img)

# 量化模型
print(f"量化前推理时间: {inference_time_before:.6f} 秒")


#### 2. 量化後的推理時間和效能測驗：

In [ ]:
import torch
import time

# 定義一個函數來測試模型的推理時間
def test_inference_time(model, img, iterations=100): 
    start_time = time.time() # 計時開始
    with torch.no_grad(): # 禁止梯度計算來加速推理
        for _ in range(iterations): # 假設我們測試 100 次推理
            _ = model(img) # 進行推理
    end_time = time.time() # 計時結束
    return (end_time - start_time) / iterations # 計算平均推理時間

# 模擬一張 640x640 大小的隨機圖像
img = torch.randn(1, 3, 640, 640)  # 

# 加載你的模型
model = torch.load('path_to_your_model.pt')

# 測試模型的推理時間
quantized_model = torch.quantization.quantize_dynamic(
    model,  # 要量化的模型
    {torch.nn.Linear},  # 要量化的層類型
    dtype=torch.qint8  # 量化的數據類型
)

# 測試量化後的推理時間
inference_time_after = test_inference_time(quantized_model, img)

# 輸出結果
print(f"量化後推理時間: {inference_time_after:.6f} 秒")


**為什麼要進行量化？**
量化的目的是減少模型的計算成本和內存佔用，同時在保持合理準確率的情況下加速推理。量化技術特別適合應用於資源受限的環境（例如移動設備或嵌入式系統），這樣可以在保持模型性能的前提下大幅減少模型的資源需求。

---

### 第四步：混合精度訓練（Mixed Precision Training）

**混合精度訓練** 是一種通過將浮點數計算混合使用 **32 位（FP32）** 和 **16 位（FP16）** 浮點數的技術來提高訓練效率的方法。這種技術特別適合使用 GPU 訓練大型深度學習模型，因為它能減少計算負擔並加速訓練過程，同時節省記憶體。

#### 為什麼使用混合精度訓練？
1. **加速訓練速度：** 通過使用 FP16（16 位浮點數），計算的精度略微降低，但速度明顯加快，尤其在 GPU 上進行的計算中效果更為顯著。
2. **降低記憶體佔用：** 由於 FP16 佔用的記憶體比 FP32 小，可以在同樣的記憶體資源下訓練更大的模型或使用更大的 batch size。
3. **保持數值穩定性：** 混合精度訓練仍然保留部分關鍵操作的 FP32 精度來避免數值溢出或精度下降，這意味著在加速的同時仍保持模型的數值穩定。

#### 如何實現混合精度訓練？
PyTorch 提供了 **`torch.cuda.amp`**（自動混合精度）這個工具來輕鬆實現混合精度訓練。下面是實現步驟：

### 1. 訓練前準備

確保你有合適的 GPU 環境，因為混合精度訓練通常需要支援 **NVIDIA Tensor Cores** 的硬體設備。

### 2. 修改訓練循環
在你的訓練循環中引入 `torch.cuda.amp.autocast()` 和 `GradScaler` 來實現自動混合精度。

#### 關鍵步驟解釋：
1. **`autocast()`**：這是一個上下文管理器，在它的作用範圍內，模型的前向傳播會自動切換到適合的精度（FP16 或 FP32），以獲得更高的運算效率。
2. **`GradScaler`**：用來縮放損失並防止在 FP16 計算中可能出現的數值溢出。`GradScaler` 會根據每一步的損失自動調整縮放因子，並在必要時調整計算精度。

#### 示例程式碼：

In [ ]:
model = MyModel().cuda()  # 將模型移動到 GPU 上，假設有 CUDA 支援
optimizer = optim.Adam(model.parameters())  # 使用 Adam 優化器
loss_fn = nn.CrossEntropyLoss()  # 定義損失函數

# 加載你的模型和數據
model = MyModel().cuda()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

# 創建 GradScaler 用來縮放梯度
scaler = GradScaler()

# 模擬的訓練循環
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.cuda(), labels.cuda()  # 將輸入和標籤移動到 GPU 上

        # 前向傳播，開啟 autocast 來進行混合精度計算
        with autocast():  # 啟動自動混合精度
            outputs = model(inputs)  # 前向傳播
            loss = loss_fn(outputs, labels)  # 計算損失

        # 反向傳播和優化，使用 GradScaler
        scaler.scale(loss).backward()  # 縮放損失值並計算梯度
        scaler.step(optimizer)  # 更新模型參數
        scaler.update()  # 更新 scaler

        # 清空梯度
        optimizer.zero_grad()  # 清空上一次迭代的梯度

    print(f"Epoch {epoch} finished")

使用 PyTorch 的 `torch.save()` 函數來保存訓練後的模型權重。通常我們會保存兩種類型的檔案：

1. **僅保存模型的參數（權重）**：這是最常見的保存方式，適合之後在相同模型結構下進行推理或繼續訓練。
2. **保存整個模型**：這包括模型的架構和權重，適合在不再定義模型架構的情況下重用。

### 1. 保存模型權重

你可以使用 `torch.save()` 將模型的權重保存到指定的檔案路徑。以下是具體的程式碼：

In [ ]:
import torch

# 保存模型的參數（權重）
model_path = r'D:\Learning_Python\30-Day_AI_Deep_Learning_Plan\yolov5-master\weights\my_model_weights.pth'
torch.save(model.state_dict(), model_path)

print(f"模型權重已保存至: {model_path}")

### 2. 保存整個模型

如果你需要保存整個模型，包括模型結構和權重，那可以這樣做：

In [ ]:
# 保存整個模型
model_path = r'D:\Learning_Python\30-Day_AI_Deep_Learning_Plan\yolov5-master\weights\my_full_model.pth'
torch.save(model, model_path)

print(f"完整模型已保存至: {model_path}")

### 3. 測試混合精度訓練效能
在進行測試時，你可以使用混合精度來進行推理。與訓練類似，你可以在推理時使用 `autocast()`。

In [ ]:
#測試模型的推理時間(不使用混合精度)

import torch

# 假設已經有加載好的模型
model = torch.hub.load('ultralytics/yolov5', 'custom', path='path_to_your_model_weights.pt')

# 模型切換到評估模式（推理模式）
model.eval()

# 模擬一些輸入資料（假設輸入是一張圖像）
# 你可以用實際的圖像處理庫 (例如 OpenCV 或 PIL) 來處理輸入，這裡假設 inputs 是處理好的張量
inputs = torch.randn(1, 3, 640, 640)  # 模擬一個大小為 640x640 的輸入圖像

# 在推理過程中不需要計算梯度，不使用混合精度推理
with torch.no_grad():
    outputs = model(inputs)  # 直接進行推理，無需 autocast

# 打印推理結果（根據具體模型來處理輸出結果）
print(outputs)

In [ ]:
#測試模型的推理時間(使用混合精度)

import torch
from torch.cuda.amp import autocast

# 假設已經有加載好的模型
model = torch.hub.load('ultralytics/yolov5', 'custom', path='path_to_your_model_weights.pt')

# 模型切換到評估模式（推理模式）
model.eval()

# 模擬一些輸入資料（假設輸入是一張圖像）
# 你可以用實際的圖像處理庫 (例如 OpenCV 或 PIL) 來處理輸入，這裡假設 inputs 是處理好的張量
inputs = torch.randn(1, 3, 640, 640)  # 模擬一個大小為 640x640 的輸入圖像

# 在推理過程中不需要計算梯度，並使用自動混合精度進行推理
with torch.no_grad():
    with autocast():  # 使用混合精度推理
        outputs = model(inputs)

# 打印推理結果（根據具體模型來處理輸出結果）
print(outputs)

### 4. 優化效果觀察
- **訓練速度提升：** 在大型資料集或模型上，混合精度能顯著減少訓練時間，特別是在使用 GPU 訓練時。
- **節省記憶體：** FP16 會減少所需記憶體，這讓你可以使用更大的 batch size 或訓練更複雜的模型。
- **模型準確率保持穩定：** 雖然使用了較低精度的 FP16，但透過保留關鍵計算（如權重更新）在 FP32 下進行，模型的準確性應該不會顯著下降。

### 總結：
混合精度訓練是一種提高訓練效率的有效技術，特別適合 GPU 訓練。它可以加速訓練過程並節省記憶體，同時保證數值穩定性。通過 `torch.cuda.amp` 進行實現，這個過程對已有的訓練流程改動很小，但效益非常明顯。接下來，你可以將這個技術應用到你的 YOLO 模型或其他大型模型上，以進一步優化推理效能和資源利用。

你可以試著在你的 YOLO 模型上應用混合精度訓練，並觀察推理時間和訓練速度的提升。

---

恭喜你完成了「模型優化與效能提升」的課程！從剪枝、量化到混合精度訓練，你已經掌握了各種提升模型效能的技術。至此，這一部分的課程可以告一段落了。

如果你感覺掌握得不錯，可以接下來考慮進行以下幾個方向的深入學習或實踐：

1. **更深入的效能測試與分析**：
   - 嘗試用更大規模的資料集來測試你優化後的模型效能，例如 COCO 資料集。
   - 探討如何在不同硬體（如 GPU、CPU）上進行推理效能測試，並深入分析資源消耗。

2. **自動化與部署**：
   - 在完成了模型優化後，下一步可以考慮將這些技術應用於實際系統中，並將這些優化後的模型部署到生產環境中。
   - 如果想進一步提升模型部署的實戰經驗，可以嘗試自動化部署，並結合模型監控技術，讓你之前學到的監控和優化流程更完整。

3. **探索新技術**：
   - 如果對效能提升的技術有更高興趣，未來可以探索如「神經結構搜索 (Neural Architecture Search, NAS)」、「分散式訓練 (Distributed Training)」等進階技術，這些都能幫助進一步提升模型的效率與效果。

目前的學習過程到這裡算是告一段落了，但學習的路上永無止境。隨時可以再回來進行更進階的探索！

---